In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

file_path ="../data/seeds_dataset.txt"

# 데이터 열 이름 예시 (실제 데이터에 맞게 조정 필요)
column_names = ['a','p','c','length of kernel', 'width of kernel', 'asymmetry coefficient','length of kernel groove','class']

df = pd.read_csv(file_path,delim_whitespace=True,header=None,names=column_names)
df

,a,p,c,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [2]:
# 결측치 확인
df.isnull().sum()

a                          0
p                          0
c                          0
length of kernel           0
width of kernel            0
asymmetry coefficient      0
length of kernel groove    0
class                      0
dtype: int64

In [4]:
df['class'].value_counts()

class
1    70
2    70
3    70
Name: count, dtype: int64

In [5]:
# x,y 분리
X=df.drop('class',axis=1)
y=df['class']
X.head()

,a,p,c,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175


In [6]:
# train, test 분리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

In [7]:

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One-hot 인코딩
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

# TensorFlow 데이터셋 생성
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
# 배치 크기 설정 및 셔플
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# 입력 레이어 정의
inputs = tf.keras.Input(shape=(X_train_scaled.shape[1],))  # 입력 데이터의 특성 수

# Dense 레이어 체인
x = tf.keras.layers.Dense(16, activation='relu')(inputs)
x = tf.keras.layers.Dropout(0.5)(x)  # 드롭아웃 추가
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)  # 드롭아웃 추가
# 출력 레이어
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)  # 클래스 수에 맞게 유닛 수 설정